# Bond Option Pricing using Various Models

Value an option on a coupon paying bond using the Hull-White, Black-Derman-Toy and Black-Karasinski model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [20]:
from financepy.finutils import *
from financepy.market.curves import *
from financepy.models.FinModelRatesBK import FinModelRatesBK
from financepy.models.FinModelRatesBDT import FinModelRatesBDT
from financepy.models.FinModelRatesHW import FinModelRatesHW
from financepy.products.bonds import *

## Set up the Discount Curve

In [21]:
settlementDate = FinDate(1, 12, 2019)

Set up discount curve

In [22]:
rate = 0.10
dcType = FinDayCountTypes.THIRTY_360_BOND
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
discountCurve = FinDiscountCurveFlat(settlementDate, rate, fixedFreq, dcType)

## Set up the Bond Option

First create the bond

In [23]:
issueDate = FinDate(1,12,2018)
maturityDate = issueDate.addTenor("3Y")
coupon = 0.10
frequencyType = FinFrequencyTypes.SEMI_ANNUAL
accrualType = FinDayCountTypes.THIRTY_360_BOND
bond = FinBond(issueDate, maturityDate, coupon, frequencyType, accrualType)

Let's first price the bond on the libor curve

In [24]:
cp = bond.cleanPriceFromDiscountCurve(settlementDate, discountCurve)
fp = bond.fullPriceFromDiscountCurve(settlementDate, discountCurve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Full  Price: %9.3f"% fp)

Fixed Income Clean Price:   100.000
Fixed Income Full  Price:   105.000


In [25]:
settlementDate

01-DEC-2019

In [26]:
bond.printFlows(settlementDate)

 01-DEC-2019          5.00 
 01-JUN-2020          5.00 
 01-DEC-2020          5.00 
 01-JUN-2021          5.00 
 01-DEC-2021        105.00 


# Create the Bond Options

Then define and create the option

In [27]:
expiryDate = settlementDate.addTenor("18m")
strikePrice = 95.0
faceAmount = 100.0

In [28]:
europeanCallBondOption = FinBondOption(bond, expiryDate, strikePrice, faceAmount, FinOptionTypes.EUROPEAN_CALL)
americanCallBondOption = FinBondOption(bond, expiryDate, strikePrice, faceAmount, FinOptionTypes.AMERICAN_CALL)

In [29]:
europeanPutBondOption = FinBondOption(bond, expiryDate, strikePrice, faceAmount, FinOptionTypes.EUROPEAN_PUT)
americanPutBondOption = FinBondOption(bond, expiryDate, strikePrice, faceAmount, FinOptionTypes.AMERICAN_PUT)

Consider the forward bond price.

In [30]:
cp = bond.cleanPriceFromDiscountCurve(expiryDate, discountCurve)
fp = bond.fullPriceFromDiscountCurve(expiryDate, discountCurve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Full  Price: %9.3f"% fp)

Fixed Income Clean Price:   100.000
Fixed Income Full  Price:   105.000


In [31]:
df = discountCurve.df(expiryDate)
callIntrinsic = max(cp - strikePrice, 0) * df
putIntrinsic = max(strikePrice - cp, 0) * df

In [32]:
callIntrinsic

4.319187992657368

In [33]:
putIntrinsic

0.0

## Tree Models

Set the model parameters, start with zero vol.

In [34]:
numSteps = 200

In [35]:
sigma = 0.00001
a = 0.1
modelHW = FinModelRatesHW(sigma*rate, a, numSteps)
modelBK = FinModelRatesBK(sigma, a, numSteps)
modelBDT = FinModelRatesBDT(sigma, numSteps)

Check some limits - the call option should be worth the intrinsic.

In [36]:
ec = europeanCallBondOption.value(settlementDate, discountCurve, modelHW)
ac = americanCallBondOption.value(settlementDate, discountCurve, modelHW)
print("European Call Value: %9.5f" % ec)
print("American Call Value: %9.5f" % ac)

European Call Value:   4.30649
American Call Value:   4.96842


The put option too.

In [37]:
ep = europeanPutBondOption.value(settlementDate, discountCurve, modelHW)
ap = americanPutBondOption.value(settlementDate, discountCurve, modelHW)
print("European Put Value: %9.5f" % ep)
print("American Put Value: %9.5f" % ap)

European Put Value:   0.00000
American Put Value:   0.00000


Now we turn on the volatility.

In [38]:
sigma = 0.20
a = 0.1
modelHW = FinModelRatesHW(sigma*rate, a, numSteps)
modelBK = FinModelRatesBK(sigma, a, numSteps)
modelBDT = FinModelRatesBDT(sigma, numSteps)

In [39]:
ec = europeanCallBondOption.value(settlementDate, discountCurve, modelHW)
ac = americanCallBondOption.value(settlementDate, discountCurve, modelHW)
print("European Call Value: %9.5f" % ec)
print("American Call Value: %9.5f" % ac)

European Call Value:   4.30650
American Call Value:   5.26581


In [40]:
ep = europeanPutBondOption.value(settlementDate, discountCurve, modelHW)
ap = americanPutBondOption.value(settlementDate, discountCurve, modelHW)
print("European Put Value: %9.5f" % ep)
print("American Put Value: %9.5f" % ap)

European Put Value:   0.00000
American Put Value:   0.00454


Copyright (c) Dominic O'Kane 2020